In [1]:
import sys
sys.path.append('../')

import pandas as pd
from pyseus import basic_processing as ip
from pyseus import primary_analysis as pa
from pyseus import validation_analysis as val

In [2]:
root = '../data/20210719_Infected_Cell_All_Samples/'
analysis = '20210726/'


In [3]:
pvals = pd.read_pickle(root + analysis + 'pval_tables.pkl')

In [4]:
standard = pvals.standard_hits_table

In [5]:
standard

values,experiment,target,prey,protein_ids,pvals,enrichment
0,20210719,RTN4-infected,MT-ND1;ND1;NADH1;NDI;ndh1,P03886;W8D6M0;R4I9W6;T1SW94;T1Q620;S4SPU5;R9YB...,1.011429,1.366500
1,20210719,RTN4-infected,ND6;NADH6;COX1;MT-ND6,A0A0A0PZ76;T1Q6P2;V9JR71;V9JDL3;V9JBQ6;V9J6I9;...,1.890862,3.098406
2,20210719,RTN4-infected,MT-CO1;COX1;coxI;COI;CO1,P00395;A0A096X0J8;A0A096WPC6;A0A096WLG7;A0A096...,0.483747,0.607438
3,20210719,RTN4-infected,COX3;COIII;CO3;coxIII;mtCOX3;MT-CO3,A0A075X871;H9QZW3;H9QNT9;H9SHR2;H9QP82;H9QQ96;...,0.385209,0.877600
4,20210719,RTN4-infected,FLJ10292;MAGOHB,A0A023T6R1;Q96A72;F5H6N1;F5H6P7;F5H3U9;F5H124,0.373741,-0.675300
...,...,...,...,...,...,...
5494,20210719,TOMM20-infected,HEL-S-31;MAP4K4,V9HWH3;E7ENQ1;H7C360;G5E948;B7Z3V5;G3XAA2,2.532589,-3.527777
5495,20210719,TOMM20-infected,LMNA,W8QEH3,0.574009,-4.566592
5496,20210719,TOMM20-infected,HLA-A,X2L7S8;Q29947,3.016742,-2.042000
5497,20210719,TOMM20-infected,RPL10,X5D2T3;X1WI28;P27635;B8A6G2;X5D2W5;A6QRI9,1.087423,0.308600


In [6]:
standard['experiment'] = standard['target'].apply(lambda x: x.split('-')[1])
standard['target'] = standard['target'].apply(lambda x: x.split('-')[0])

In [7]:
standard

values,experiment,target,prey,protein_ids,pvals,enrichment
0,infected,RTN4,MT-ND1;ND1;NADH1;NDI;ndh1,P03886;W8D6M0;R4I9W6;T1SW94;T1Q620;S4SPU5;R9YB...,1.011429,1.366500
1,infected,RTN4,ND6;NADH6;COX1;MT-ND6,A0A0A0PZ76;T1Q6P2;V9JR71;V9JDL3;V9JBQ6;V9J6I9;...,1.890862,3.098406
2,infected,RTN4,MT-CO1;COX1;coxI;COI;CO1,P00395;A0A096X0J8;A0A096WPC6;A0A096WLG7;A0A096...,0.483747,0.607438
3,infected,RTN4,COX3;COIII;CO3;coxIII;mtCOX3;MT-CO3,A0A075X871;H9QZW3;H9QNT9;H9SHR2;H9QP82;H9QQ96;...,0.385209,0.877600
4,infected,RTN4,FLJ10292;MAGOHB,A0A023T6R1;Q96A72;F5H6N1;F5H6P7;F5H3U9;F5H124,0.373741,-0.675300
...,...,...,...,...,...,...
5494,infected,TOMM20,HEL-S-31;MAP4K4,V9HWH3;E7ENQ1;H7C360;G5E948;B7Z3V5;G3XAA2,2.532589,-3.527777
5495,infected,TOMM20,LMNA,W8QEH3,0.574009,-4.566592
5496,infected,TOMM20,HLA-A,X2L7S8;Q29947,3.016742,-2.042000
5497,infected,TOMM20,RPL10,X5D2T3;X1WI28;P27635;B8A6G2;X5D2W5;A6QRI9,1.087423,0.308600


In [66]:
def find_differential(standard_table, target, curvature, offset):
    """

    """
    standard = standard_table.copy()
    standard = standard[standard['target']==target]
    vali = val.Validation(standard, '', '')
    vali.static_fdr(curvature=curvature, offset=offset)
    hits = vali.interaction_table

    ints = hits[hits['interaction']==True]
    no_ints = hits[hits['interaction']==False]
    no_infected = no_ints[no_ints['experiment']=='infected'][['protein_ids', 'enrichment']].rename(columns={'enrichment':'!enrichment'})
    no_uninfected = no_ints[no_ints['experiment']=='uninfected'][['protein_ids', 'enrichment']].rename(columns={'enrichment':'!enrichment'})

    ints.drop_duplicates(subset='protein_ids', keep=False, inplace=True)
    infected = ints[ints['experiment']=='infected']
    uninfected = ints[ints['experiment']=='uninfected']   

    inf_merge = infected.merge(no_uninfected, on='protein_ids', how='left')
    uninf_merge = uninfected.merge(no_infected, on='protein_ids', how='left')

    final = pd.concat([inf_merge, uninf_merge]).drop(columns=['interaction'])
    final['enrichment differential'] = final['enrichment'] - final['!enrichment']
    final.sort_values(by=['experiment', 'enrichment differential'], ascending=[True, False], inplace=True)
    final.reset_index(drop=True, inplace=True)
    return final



In [83]:
hits = find_differential(standard, 'TOMM20', 4, 4)

In [84]:
hits

values,experiment,target,prey,protein_ids,pvals,enrichment,fdr,!enrichment,enrichment differential
0,infected,TOMM20,N,OC43_N,6.143446,17.687100,"[4, 4]",0.480700,17.206400
1,infected,TOMM20,M,OC43_M,2.851444,18.315000,"[4, 4]",1.811600,16.503400
2,infected,TOMM20,HE,OC43_HE,4.631506,9.099500,"[4, 4]",-3.241900,12.341400
3,infected,TOMM20,S,OC43_S,5.461839,13.425500,"[4, 4]",3.288900,10.136600
4,infected,TOMM20,PLPRO,OC43_ORF1A_PLpro,3.909276,10.381100,"[4, 4]",0.451200,9.929900
5,infected,TOMM20,3CL,OC43_ORF1A_3CL,4.111270,6.503000,"[4, 4]",-1.250589,7.753589
6,infected,TOMM20,NS2,OC43_ORF1A_NS2,2.943877,10.396400,"[4, 4]",2.824800,7.571600
7,infected,TOMM20,NS10,OC43_ORF1A_NS10,3.451683,6.884525,"[4, 4]",-0.247994,7.132518
8,infected,TOMM20,HEL,OC43_ORF1AB_HEL;B9EIS3;Q9BXT6,5.157492,5.309651,"[4, 4]",-1.636949,6.946600
9,infected,TOMM20,NS8,OC43_ORF1A_NS8,3.974236,7.941030,"[4, 4]",1.145854,6.795176


In [85]:
hits.to_csv(root+analysis+'TOMM20_interaction_differential.csv')

In [35]:
ints = hits[hits['interaction']==True]
no_ints = hits[hits['interaction']==False]
no_infected = no_ints[no_ints['experiment']=='infected'][['protein_ids', 'enrichment']].rename(columns={'enrichment':'!enrichment'})
no_uninfected = no_ints[no_ints['experiment']=='uninfected'][['protein_ids', 'enrichment']].rename(columns={'enrichment':'!enrichment'})

In [36]:
ints.drop_duplicates(subset='protein_ids', keep=False, inplace=True)
infected = ints[ints['experiment']=='infected']
uninfected = ints[ints['experiment']=='uninfected']

In [20]:
infected

values,experiment,target,prey,protein_ids,pvals,enrichment,fdr,interaction
2105,infected,LAMP1,SH2D3C,A8K9K3;Q8N5H7;A8K7M6;Q5JU32;B4DG32;B3KUE2;H0Y5X5,3.739095,5.204000,"[4, 4]",True
2864,infected,LAMP1,MDK,E9PPJ5;E9PLM6;P21741,6.002509,5.467900,"[4, 4]",True
3378,infected,LAMP1,HE,OC43_HE,3.673382,6.373500,"[4, 4]",True
3379,infected,LAMP1,M,OC43_M,2.632031,19.521100,"[4, 4]",True
3380,infected,LAMP1,N,OC43_N,6.148701,17.351100,"[4, 4]",True
3384,infected,LAMP1,NS2,OC43_ORF1A_NS2,2.879888,8.668300,"[4, 4]",True
3388,infected,LAMP1,NS8,OC43_ORF1A_NS8,3.931465,6.711330,"[4, 4]",True
3390,infected,LAMP1,PLPRO,OC43_ORF1A_PLpro,3.603831,9.302500,"[4, 4]",True
3396,infected,LAMP1,S,OC43_S,4.263428,11.396600,"[4, 4]",True
4229,infected,LAMP1,COA6,Q5JTJ3,4.769038,5.504288,"[4, 4]",True


In [37]:
inf_merge = infected.merge(no_uninfected, on='protein_ids', how='left')
uninf_merge = uninfected.merge(no_infected, on='protein_ids', how='left')

In [42]:
final = pd.concat([inf_merge, uninf_merge]).drop(columns=['interaction']).reset_index(drop=True)
final['enrichment differential'] = final['enrichment'] - final['!enrichment']

In [43]:
final

values,experiment,target,prey,protein_ids,pvals,enrichment,fdr,!enrichment,enrichment differential
0,infected,LAMP1,SH2D3C,A8K9K3;Q8N5H7;A8K7M6;Q5JU32;B4DG32;B3KUE2;H0Y5X5,3.739095,5.204000,"[4, 4]",2.613000,2.591000
1,infected,LAMP1,MDK,E9PPJ5;E9PLM6;P21741,6.002509,5.467900,"[4, 4]",2.414000,3.053900
2,infected,LAMP1,HE,OC43_HE,3.673382,6.373500,"[4, 4]",-1.652837,8.026337
3,infected,LAMP1,M,OC43_M,2.632031,19.521100,"[4, 4]",2.590700,16.930400
4,infected,LAMP1,N,OC43_N,6.148701,17.351100,"[4, 4]",-1.257300,18.608400
5,infected,LAMP1,NS2,OC43_ORF1A_NS2,2.879888,8.668300,"[4, 4]",1.369700,7.298600
6,infected,LAMP1,NS8,OC43_ORF1A_NS8,3.931465,6.711330,"[4, 4]",0.749638,5.961691
7,infected,LAMP1,PLPRO,OC43_ORF1A_PLpro,3.603831,9.302500,"[4, 4]",2.170100,7.132400
8,infected,LAMP1,S,OC43_S,4.263428,11.396600,"[4, 4]",-0.462200,11.858800
9,infected,LAMP1,COA6,Q5JTJ3,4.769038,5.504288,"[4, 4]",0.213333,5.290955
